In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
import io
import datetime
from tqdm import tqdm
import base64

In [2]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly','https://www.googleapis.com/auth/drive',"https://www.googleapis.com/auth/calendar.readonly"]

In [3]:
def Authenticate():
  
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  return creds

creds = Authenticate()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1001284421848-9utttomg5ds10bd9p2ufabrecsu2cb0m.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A43469%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=41SOcVoiVBTNpPYw24YcjiHUvyyXf7&access_type=offline


Gtk-Message: 20:46:37.407: Failed to load module "canberra-gtk-module"
Gtk-Message: 20:46:37.409: Failed to load module "canberra-gtk-module"


Opening in existing browser session.


In [4]:
def get_message_body(msg):
    payload = msg.get("payload", {})
    if "parts" in payload:
        for part in payload["parts"]:
            if part["mimeType"] == "text/plain":
                data = part["body"]["data"]
                return base64.urlsafe_b64decode(data).decode("utf-8")
    else:
        data = payload.get("body", {}).get("data")
        if data:
            return base64.urlsafe_b64decode(data).decode("utf-8")
    return None


class gmail:
    def __init__(self, credentials):

        try:
            self.service = build("gmail", "v1", credentials=credentials)
        except:
            self.service= None
            print("Unable to make connection with gmail")
    def Inbox(self,results: int = 5):
        try:

            print("Fetching Message ID's...", end="\r")
            results = (self.service.users().messages().list(userId="me", labelIds=["INBOX"], maxResults=results).execute())
            messages = results.get("messages", [])

            if not messages:
              print("No messaged found in inbox.")
              return "No messaged found in inbox."
            
            print("Fetching Complete Messages...",end="\r")
            detailed_messages=[]
            for m in tqdm(messages):
                msg_id=m['id']
                msg_detail = self.service.users().messages().get(
                    userId="me",
                    id=msg_id,
                    format="full",
                ).execute()
                payload = msg_detail.get("payload", {})
                headers = payload.get("headers", [])
                body = get_message_body(msg_detail)

                details = {h["name"]: h["value"] for h in headers}
                detailed_messages.append({
                    "id": msg_id,
                    "from": details.get("From"),
                    "subject": details.get("Subject"),
                    "date": details.get("Date"),
                    "body": body
                })

            return detailed_messages

        except Exception as error:
            # TODO(developer)
            return f"Error: {error}"
        

    def search(self,query: str,results: int = 5):
        try:

            print("Searching for Mails", end="\r")
            results = (self.service.users().messages().list(userId="me", labelIds=["INBOX"], maxResults=results, q=query).execute())
            messages = results.get("messages", [])

            if not messages:
              print("No messaged found in inbox.")
              return "No messaged found in inbox."
            
            print("Fetching Complete Messages...",end="\r")
            detailed_messages=[]
            for m in tqdm(messages):
                msg_id=m['id']
                msg_detail = self.service.users().messages().get(
                    userId="me",
                    id=msg_id,
                    format="full",
                ).execute()
                payload = msg_detail.get("payload", {})
                headers = payload.get("headers", [])
                body = get_message_body(msg_detail)


                details = {h["name"]: h["value"] for h in headers}
                detailed_messages.append({
                    "id": msg_id,
                    "from": details.get("From"),
                    "subject": details.get("Subject"),
                    "date": details.get("Date"),
                    "body": body
                })

            return detailed_messages

        except Exception as error:
            # TODO(developer)
            return f"Error: {error}"


In [5]:
mailbox=gmail(credentials=creds)
mails=mailbox.Inbox()
for mail in mails:
    print(mail['body'],"\n\n\n\n\n\n\n\n\n\n\n")

100%|██████████| 5/5 [00:03<00:00,  1.45it/s]

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office">
<head>
<!-- MonitorCampaignID: email_digest -->

<!--[if gte mso 9]>
<xml>
<o:OfficeDocumentSettings>
<o:AllowPNG/>
<o:PixelsPerInch>96</o:PixelsPerInch>
</o:OfficeDocumentSettings>
</xml>
<![endif]-->
<meta http-equiv="Content-type" content="text/html; charset=utf-8" />
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1" />
<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<meta name="format-detection" content="date=no" />
<meta name="format-detection" content="address=no" />
<meta name="format-detection" content="telephone=no" />
<meta name="x-apple-disable-message-reformatting" />
<!--[if !mso]><!-->
<link rel="preconnect" href="https://fonts.gstatic.com" />
<link href="https://fonts.googleapi

In [6]:
mails=mailbox.search("Instagram")
for mail in mails:
    print(mail['body'],"\n\n\n\n\n\n\n\n\n\n\n")

100%|██████████| 5/5 [00:03<00:00,  1.39it/s]



Canva  ( https://l.engage.canva.com/ss/c/u001.Note3it1gaozj0PIggVG9tplar3juom3GwdJknmjC202CX1mRbM66MKwrv-9ayTO/4j8/Ze19IhprREy9f5F_3FDjIA/t0/h001.mgqb-dKA9Qrfi9g8K6e0YZ_hUP4d55oSyC0pl-Q27Jk )
Pro           Canva  
( https://l.engage.canva.com/ss/c/u001.Note3it1gaozj0PIggVG9tplar3juom3GwdJknmjC22DOWnwDwy0dx_FrPz3zatH/4j8/Ze19IhprREy9f5F_3FDjIA/t1/h001.O_Zi825LblmuH6vyw_65pa0ubL_TFntzG1ypTc9PIXo )
    
( https://l.engage.canva.com/ss/c/u001.Note3it1gaozj0PIggVG9jSYjQmDyWZrqxUXjHgaLPLrZ8tIb_ozLBO7iNkCjuk7u8v3ndXBmFTNN9lyll-yZPY0ZOkbzv7uLfLbQBHvotwqxNoV2RraAUeZkX_-5lIG/4j8/Ze19IhprREy9f5F_3FDjIA/t2/h001.t8zvsZfqOOmVT_M-r1PI8VpDkEjbzSN6ZbHt-qmYcHk )
         
( https://l.engage.canva.com/ss/c/u001.Note3it1gaozj0PIggVG9tplar3juom3GwdJknmjC21NxW-qjDT3dsSNkidPV_NV/4j8/Ze19IhprREy9f5F_3FDjIA/t3/h001.h_cwcOwWTG6M0OFaaZYMYx6soZA8OavJhSX6CaRuQsc )





(  ) (
 )


(
?lid=0s5qey4kiz94 )





 (
?lid=cqopwdvvjua0 )
( &lid=6e2qo1xk5nnk )





 (
&lid=g6pfqgrpm2xr )
(
?lid=a4h2nbv7piki )





 (
?li

In [7]:
class GoogleCalendar:
    def __init__(self, credentials):
        try:
            self.service = build("calendar", "v3", credentials=credentials)
            
        except:
            self.service= None
            print("Unable to make connection with gmail")

    def upcoming_events(self, max_results=10):
        now = datetime.datetime.now(datetime.UTC).isoformat()
        results = self.service.events().list(
            calendarId="primary",
            timeMin=now,
            maxResults=max_results,
            singleEvents=True,
            orderBy="startTime"
        ).execute()
        events = results.get("items", [])

        formatted = []
        for event in events:
            start = event["start"].get("dateTime", event["start"].get("date"))
            formatted.append({
                "start": start,
                "summary": event.get("summary", "(No Title)"),
                "id": event.get("id"),
                "location": event.get("location"),
                "description": event.get("description"),
            })
        return formatted
    
    def search_events(self, query, max_results=10, days_ahead=30):
        now = datetime.datetime.now(datetime.UTC).isoformat()
        future = (datetime.datetime.now(datetime.UTC) + datetime.timedelta(days=days_ahead)).isoformat()

        results = self.service.events().list(
            calendarId="primary",
            q=query,
            timeMin=now,
            timeMax=future,
            maxResults=max_results,
            singleEvents=True,
            orderBy="startTime"
        ).execute()

        events = results.get("items", [])
        return [
            {
                "id": e["id"],
                "start": e["start"].get("dateTime", e["start"].get("date")),
                "summary": e.get("summary", "(No Title)"),
                "location": e.get("location"),
            }
            for e in events
        ]
    
    
    # Can do if got time
    def create_event(self, summary, start_time, end_time, timezone="UTC"):
        event = {
            "summary": summary,
            "start": {"dateTime": start_time, "timeZone": timezone},
            "end": {"dateTime": end_time, "timeZone": timezone},
        }
        return self.service.events().insert(calendarId="primary", body=event).execute()


In [8]:
calender=GoogleCalendar(credentials=creds)
upcoming_events=calender.upcoming_events()
print(upcoming_events)

[{'start': '2025-08-22T09:00:00+05:30', 'summary': 'ESO213 (L12)', 'id': 'a918e134995f4b40bbd94752781b6a9e_20250822T033000Z', 'location': None, 'description': None}, {'start': '2025-08-22T11:00:00+05:30', 'summary': 'MSO205 Lec (L18)', 'id': '8508a789894642b0a2df02ab3f926174_20250822T053000Z', 'location': None, 'description': None}, {'start': '2025-08-22T17:00:00+05:30', 'summary': 'MTH201 Lec (L13)', 'id': 'e7525ea0cfd44379aefefc46211303a7_20250822T113000Z', 'location': None, 'description': None}, {'start': '2025-08-25T08:00:00+05:30', 'summary': 'ES208 (L11)', 'id': '2bdaae6f9479475f8b42d28ff17826c0_20250825T023000Z', 'location': None, 'description': None}, {'start': '2025-08-25T09:00:00+05:30', 'summary': 'ESO213 (L12)', 'id': 'a918e134995f4b40bbd94752781b6a9e_20250825T033000Z', 'location': None, 'description': None}, {'start': '2025-08-25T11:00:00+05:30', 'summary': 'MSO205 Lec (L18)', 'id': '8508a789894642b0a2df02ab3f926174_20250825T053000Z', 'location': None, 'description': None}

In [9]:
next_class=calender.search_events("ES208")
print(next_class)

[{'id': '2bdaae6f9479475f8b42d28ff17826c0_20250825T023000Z', 'start': '2025-08-25T08:00:00+05:30', 'summary': 'ES208 (L11)', 'location': None}, {'id': '2bdaae6f9479475f8b42d28ff17826c0_20250827T023000Z', 'start': '2025-08-27T08:00:00+05:30', 'summary': 'ES208 (L11)', 'location': None}, {'id': '2bdaae6f9479475f8b42d28ff17826c0_20250828T023000Z', 'start': '2025-08-28T08:00:00+05:30', 'summary': 'ES208 (L11)', 'location': None}, {'id': '2bdaae6f9479475f8b42d28ff17826c0_20250901T023000Z', 'start': '2025-09-01T08:00:00+05:30', 'summary': 'ES208 (L11)', 'location': None}, {'id': '2bdaae6f9479475f8b42d28ff17826c0_20250903T023000Z', 'start': '2025-09-03T08:00:00+05:30', 'summary': 'ES208 (L11)', 'location': None}, {'id': '2bdaae6f9479475f8b42d28ff17826c0_20250904T023000Z', 'start': '2025-09-04T08:00:00+05:30', 'summary': 'ES208 (L11)', 'location': None}, {'id': '2bdaae6f9479475f8b42d28ff17826c0_20250908T023000Z', 'start': '2025-09-08T08:00:00+05:30', 'summary': 'ES208 (L11)', 'location': None}

In [10]:
class GoogleDrive:
    def __init__(self, credentials):
        
        try:
            self.service = build("drive", "v3", credentials=credentials)
        except:
            self.service= None
            print("Unable to make connection with gmail")

    def search_files(self, query, max_results=10):
        if not self.service:
            return []
        results = self.service.files().list(
            q= f"name contains '{query}'",
            pageSize=max_results,
            fields="files(id, name, mimeType, modifiedTime, owners)"
        ).execute()
        return results.get("files", [])


    def download_file(self, file_id, filepath="Temporary/downloaded_file"):
        try:
            file = self.service.files().get(fileId=file_id).execute()
            mime_type = file.get("mimeType")

            if mime_type.startswith("application/vnd.google-apps"):  
                
                export_mime = None
                if mime_type == "application/vnd.google-apps.document":
                    export_mime = "application/pdf"  # or 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'
                elif mime_type == "application/vnd.google-apps.spreadsheet":
                    export_mime = "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
                elif mime_type == "application/vnd.google-apps.presentation":
                    export_mime = "application/vnd.openxmlformats-officedocument.presentationml.presentation"

                if not export_mime:
                    raise Exception(f"Export not supported for mime type {mime_type}")

                request = self.service.files().export_media(fileId=file_id, mimeType=export_mime)
            else:
                
                request = self.service.files().get_media(fileId=file_id)

            fh = io.FileIO(filepath, "wb")
            downloader = MediaIoBaseDownload(fh, request)

            done = False
            while not done:
                status, done = downloader.next_chunk()
                if status:
                    print(f"Download {int(status.progress() * 100)}%.")
            print("Download complete:", filepath)

        except HttpError as error:
            print(f"An error occurred: {error}")

In [11]:
drive=GoogleDrive(credentials=creds)
rl=drive.search_files("RL")
print(rl)

[{'owners': [{'displayName': 'Vishesh Kumar Singh', 'kind': 'drive#user', 'me': True, 'permissionId': '10189920605030927697', 'emailAddress': 'nghvishesh@gmail.com', 'photoLink': 'https://lh3.googleusercontent.com/a/ACg8ocLnHVWtIEoburlrqm77ttymOZf1Tx18_-n9c0zPNSKEbkNgWgDtmQ=s64'}], 'id': '1c_DlkRN14lIH-_hmlgNYai9jBQiVXvie64lWZUKrXc8', 'name': 'Exploring Generalisation in NLP and RL via Baby-AI', 'mimeType': 'application/vnd.google-apps.document', 'modifiedTime': '2025-08-20T03:56:39.617Z'}, {'owners': [{'displayName': 'Aviral Gupta', 'kind': 'drive#user', 'me': False, 'permissionId': '17656357186386035217', 'emailAddress': 'avixavral2005@gmail.com', 'photoLink': 'https://lh3.googleusercontent.com/a-/ALV-UjW1ztcLBy8jh9Tb0htQExWc_5yxG6DPklf2XHd8gdotEzKqIY5a=s64'}], 'id': '1JaVAv3W1G70Y0AN2FKq98N-bOzLYSgid', 'name': '7_QuantRL.pdf', 'mimeType': 'application/pdf', 'modifiedTime': '2025-04-19T12:33:23.000Z'}]


In [12]:
id=rl[0]['id']
drive.download_file(id, "Temporary/downloaded_file.pdf")

Download 100%.
Download complete: Temporary/downloaded_file.pdf
